In [1]:
import numpy as np
import pandas as pd 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('/Users/taraapple/Desktop/fashion-mnist_train.csv')
print(train.shape)

(60000, 785)


In [3]:
test = pd.read_csv('/Users/taraapple/Desktop/fashion-mnist_test.csv')
print(test.shape)

(10000, 785)


In [4]:
y_train = train['label'].values
X_train = train.drop(['label'],1).values

In [5]:
y_test = test['label'].values
X_test = test.drop(['label'],1).values

In [6]:
print(y_test.shape)

(10000,)


In [7]:
print(X_test.shape)

(10000, 784)


In [8]:
print(y_train.shape)

(60000,)


In [9]:
print(X_train.shape)

(60000, 784)


In [10]:
BATCH_SIZE = 32

torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

In [11]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(784,250)
        self.linear2 = nn.Linear(250,100)
        self.linear3 = nn.Linear(100,10)
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = self.linear3(X)
        return F.log_softmax(X, dim=1)
 
mlp = MLP()
print(mlp)

MLP(
  (linear1): Linear(in_features=784, out_features=250, bias=True)
  (linear2): Linear(in_features=250, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=10, bias=True)
)


In [12]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())#,lr=0.001, betas=(0.9,0.999))
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            #print(correct)
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.item(), float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))
                

In [13]:
fit(mlp, train_loader)

Epoch : 0 [0/60000 (0%)]	Loss: 13.926872	 Accuracy:9.375%
Epoch : 0 [1600/60000 (3%)]	Loss: 1.022905	 Accuracy:55.699%
Epoch : 0 [3200/60000 (5%)]	Loss: 0.855172	 Accuracy:63.335%
Epoch : 0 [4800/60000 (8%)]	Loss: 0.655638	 Accuracy:67.943%
Epoch : 0 [6400/60000 (11%)]	Loss: 1.071517	 Accuracy:70.134%
Epoch : 0 [8000/60000 (13%)]	Loss: 1.342290	 Accuracy:71.327%
Epoch : 0 [9600/60000 (16%)]	Loss: 0.454747	 Accuracy:72.124%
Epoch : 0 [11200/60000 (19%)]	Loss: 0.514620	 Accuracy:72.917%
Epoch : 0 [12800/60000 (21%)]	Loss: 0.963072	 Accuracy:73.706%
Epoch : 0 [14400/60000 (24%)]	Loss: 0.706065	 Accuracy:74.120%
Epoch : 0 [16000/60000 (27%)]	Loss: 0.261389	 Accuracy:74.495%
Epoch : 0 [17600/60000 (29%)]	Loss: 0.479667	 Accuracy:74.904%
Epoch : 0 [19200/60000 (32%)]	Loss: 0.321822	 Accuracy:75.224%
Epoch : 0 [20800/60000 (35%)]	Loss: 0.509780	 Accuracy:75.509%
Epoch : 0 [22400/60000 (37%)]	Loss: 0.476046	 Accuracy:75.860%
Epoch : 0 [24000/60000 (40%)]	Loss: 0.725893	 Accuracy:76.236%
Epoch 

Epoch : 3 [27200/60000 (45%)]	Loss: 0.243199	 Accuracy:85.811%
Epoch : 3 [28800/60000 (48%)]	Loss: 0.388100	 Accuracy:85.797%
Epoch : 3 [30400/60000 (51%)]	Loss: 0.407592	 Accuracy:85.870%
Epoch : 3 [32000/60000 (53%)]	Loss: 0.593659	 Accuracy:85.749%
Epoch : 3 [33600/60000 (56%)]	Loss: 0.582733	 Accuracy:85.728%
Epoch : 3 [35200/60000 (59%)]	Loss: 0.392225	 Accuracy:85.769%
Epoch : 3 [36800/60000 (61%)]	Loss: 0.353324	 Accuracy:85.765%
Epoch : 3 [38400/60000 (64%)]	Loss: 0.503502	 Accuracy:85.754%
Epoch : 3 [40000/60000 (67%)]	Loss: 0.279861	 Accuracy:85.729%
Epoch : 3 [41600/60000 (69%)]	Loss: 0.355778	 Accuracy:85.747%
Epoch : 3 [43200/60000 (72%)]	Loss: 0.521414	 Accuracy:85.721%
Epoch : 3 [44800/60000 (75%)]	Loss: 0.225724	 Accuracy:85.684%
Epoch : 3 [46400/60000 (77%)]	Loss: 0.421178	 Accuracy:85.633%
Epoch : 3 [48000/60000 (80%)]	Loss: 0.476336	 Accuracy:85.639%
Epoch : 3 [49600/60000 (83%)]	Loss: 0.279259	 Accuracy:85.622%
Epoch : 3 [51200/60000 (85%)]	Loss: 0.195279	 Accuracy:

In [14]:
def evaluate(model):
#model = mlp
    correct = 0 
    for test_imgs, test_labels in test_loader:
        #print(test_imgs.shape)
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_loader)*BATCH_SIZE)))
evaluate(mlp)

Test accuracy:0.866% 


In [15]:
torch_X_train = torch_X_train.view(-1, 1,28,28).float()
torch_X_test = torch_X_test.view(-1,1,28,28).float()
print(torch_X_train.shape)
print(torch_X_test.shape)

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

torch.Size([60000, 1, 28, 28])
torch.Size([10000, 1, 28, 28])


In [16]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32,64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x),2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1,3*3*64 )
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
 
cnn = CNN()
print(cnn)

it = iter(train_loader)
X_batch, y_batch = next(it)
print(cnn.forward(X_batch).shape)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
torch.Size([32, 10])


In [17]:
fit(cnn,train_loader)

Epoch : 0 [0/60000 (0%)]	Loss: 14.301333	 Accuracy:9.375%
Epoch : 0 [1600/60000 (3%)]	Loss: 1.303137	 Accuracy:30.576%
Epoch : 0 [3200/60000 (5%)]	Loss: 1.246851	 Accuracy:44.833%
Epoch : 0 [4800/60000 (8%)]	Loss: 0.620236	 Accuracy:51.800%
Epoch : 0 [6400/60000 (11%)]	Loss: 0.897263	 Accuracy:56.063%
Epoch : 0 [8000/60000 (13%)]	Loss: 1.674600	 Accuracy:58.902%
Epoch : 0 [9600/60000 (16%)]	Loss: 0.574538	 Accuracy:60.974%
Epoch : 0 [11200/60000 (19%)]	Loss: 0.710445	 Accuracy:62.553%
Epoch : 0 [12800/60000 (21%)]	Loss: 0.880269	 Accuracy:63.848%
Epoch : 0 [14400/60000 (24%)]	Loss: 0.841996	 Accuracy:64.932%
Epoch : 0 [16000/60000 (27%)]	Loss: 0.771100	 Accuracy:65.812%
Epoch : 0 [17600/60000 (29%)]	Loss: 0.423757	 Accuracy:66.640%
Epoch : 0 [19200/60000 (32%)]	Loss: 0.605568	 Accuracy:67.138%
Epoch : 0 [20800/60000 (35%)]	Loss: 0.733347	 Accuracy:67.680%
Epoch : 0 [22400/60000 (37%)]	Loss: 0.722771	 Accuracy:68.202%
Epoch : 0 [24000/60000 (40%)]	Loss: 0.610942	 Accuracy:68.658%
Epoch 

Epoch : 3 [27200/60000 (45%)]	Loss: 0.545763	 Accuracy:80.953%
Epoch : 3 [28800/60000 (48%)]	Loss: 0.515937	 Accuracy:80.796%
Epoch : 3 [30400/60000 (51%)]	Loss: 0.434740	 Accuracy:80.892%
Epoch : 3 [32000/60000 (53%)]	Loss: 0.347564	 Accuracy:80.860%
Epoch : 3 [33600/60000 (56%)]	Loss: 0.474426	 Accuracy:80.890%
Epoch : 3 [35200/60000 (59%)]	Loss: 0.479033	 Accuracy:80.873%
Epoch : 3 [36800/60000 (61%)]	Loss: 0.864798	 Accuracy:80.824%
Epoch : 3 [38400/60000 (64%)]	Loss: 0.488219	 Accuracy:80.836%
Epoch : 3 [40000/60000 (67%)]	Loss: 0.411302	 Accuracy:80.833%
Epoch : 3 [41600/60000 (69%)]	Loss: 0.479521	 Accuracy:80.786%
Epoch : 3 [43200/60000 (72%)]	Loss: 0.593082	 Accuracy:80.827%
Epoch : 3 [44800/60000 (75%)]	Loss: 0.337090	 Accuracy:80.833%
Epoch : 3 [46400/60000 (77%)]	Loss: 0.369015	 Accuracy:80.804%
Epoch : 3 [48000/60000 (80%)]	Loss: 0.544907	 Accuracy:80.811%
Epoch : 3 [49600/60000 (83%)]	Loss: 0.475143	 Accuracy:80.809%
Epoch : 3 [51200/60000 (85%)]	Loss: 0.197563	 Accuracy:

In [18]:
evaluate(cnn)

Test accuracy:0.814% 
